# 人工神经网络作业

### 1.实验内容
基于pytorch/tensorflow/paddlepaddle框架采用神经网络(BP或者CNN)分类CIFAR10数据集

### 2. 操作步骤
(1) 安装 pytorch/tensorflow/paddlepaddle 深度学习框架实验环境
* 可以采用anaconda安装需要的pytorch环境，可参考https://pytorch.org/get-started/locally/；
* 更多pytorch的教程可以参考https://github.com/fendouai/PyTorchDocs；https://www.bookstack.cn/read/PyTorch-cn/docs-index.md；
* 安装tensorflow的教程可以参考https://www.tensorflow.org/?hl=zh-cn
*  **强烈建议**：直接在AI studio平台上利用paddlepaddle构建深度模型！！！（不需要安装任何运行环境）

(2)下载CIFAR10数据集

![](https://ai-studio-static-online.cdn.bcebos.com/703413ad5eb84fea8fb3280c2d3d0718bf6297260e7f4ba8b6d6f569a0365f4d)
* 可利用Pytorch框架中datasets接口下载cifar10数据集（它包含十个类别：‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’。CIFAR-10 中的图像尺寸为32x32，也就是RGB的3层颜色通道，每层通道内的尺寸为32*32。）或者在官网下载。
* 作业data文件夹中已加入cifar10数据集
* 可利用paddlepaddle框架的datasets库代导入数据集：
import paddle
train_dataset = paddle.vision.datasets.cifar10(mode='train', transform=transform)
test_dataset = paddle.vision.datasets.cifar10(mode='test', transform=transform)

(3) 编写模型并训练模型
* pytorch代码可以参考https://github.com/fendouai/PyTorchDocs/blob/master/SecondSection/training_a_classifier.md；（网络模型可以用AlexNet, VGG-net, ResNet等主流CNN模型）
* paddlepaddle代码可以参考
https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/beginner/quick_start_cn.html

### 3. 提交作业
* 提交代码（数据集不要提交）+运行截图（训练过程（目标函数损失+训练数据平均准确率+测试数据平均准确率）+测试数据各类别的分类准确率）
* Dealine： 大作业答辩前

In [1]:
import tensorflow as tf
from tensorflow.python.keras import layers
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# 加载数据集并进行预处理
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

#数据标准化
train_images, test_images = train_images / 255.0, test_images / 255.0

# 定义模型结构
model = keras.Sequential(
    [
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3)),  #一个卷积层，包含32个输出通道，使用3x3大小的卷积核，激活函数为ReLU，输入数据的形状为(32, 32, 3)。
        layers.MaxPooling2D((2, 2)), #一个最大池化层，使用2x2大小的池化核。
        layers.Conv2D(64, (3, 3), activation="relu"),#一个卷积层，包含64个输出通道，使用3x3大小的卷积核，激活函数为ReLU。
        layers.MaxPooling2D((2, 2)),#一个最大池化层，使用2x2大小的池化核。
        layers.Conv2D(128, (3, 3), activation="relu"),#一个卷积层，包含128个输出通道，使用3x3大小的卷积核，激活函数为ReLU
        layers.Flatten(),#一个Flatten层，用于将卷积层输出的特征图展开成一维向量。
        layers.Dense(64, activation="relu"),#一个全连接层，包含64个隐藏单元，激活函数为ReLU。
        layers.Dense(10),#一个全连接层，包含10个输出单元，用于输出模型的分类结果。
    ]
)

# 编译模型
model.compile(optimizer='adam',#优化器：用于在反向传播过程中更新神经网络中的各个参数，从而不断优化模型的输出结果。在这里，我们使用Adam优化器，并通过learning_rate参数指定了学习率为0.001。
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),#损失函数：用于计算模型预测结果与真实标签之间的差距。在这里，我们使用交叉熵损失函数(CategoricalCrossentropy)，
    #并将from_logits参数设置为True，表示模型的输出是未经过softmax函数转换的原始数据，需要在损失计算时进行转换。
              metrics=['accuracy'])#评价指标：用于度量模型的性能表现。在这里，我们使用分类准确率(accuracy)作为评价指标，该指标会在每轮训练结束后自动计算并输出。

# 训练模型
#epochs：指定模型进行训练时要遍历整个训练数据集的次数。例如，epochs=10表示模型会遍历整个数据集10次，进行10次前向传播和反向传播过程。
#validation_split：用于划分出一部分原始训练集作为验证集。例如，validation_split=0.1表示将原始训练集中10%的数据作为验证集，剩余90%的数据作为训练集。在每个epoch结束后，会使用验证集检查模型的性能表现，并输出验证损失和评价指标等信息。
#batch_size：指定在进行梯度下降时每个批次包含的样本数量。例如，batch_size=128表示每个批次有128个样本被用来更新模型参数。较大的batch_size可以加速计算速度，但也可能导致模型的泛化能力下降，而较小的batch_size则可以提高模型的泛化能力，但也可能导致计算速度变慢。
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

# 在测试集上评估模型
test_loss, test_acc = model.evaluate(test_images,  test_labels)
print('Test accuracy:', test_acc)

# 绘制准确率和损失函数变化曲线
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# 在测试集上进行预测
pred = model.predict(test_images)

# 计算每个类别的分类准确率
from sklearn.metrics import classification_report

target_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                'dog', 'frog', 'horse', 'ship', 'truck']
print(classification_report(test_labels, pred.argmax(axis=1), target_names=target_names))



Cannot run import tensorflow because of system compatibility. AI Studio prepared an entire environment based on PaddlePaddle already. Please use PaddlePaddle to build your own model or application.


![](https://ai-studio-static-online.cdn.bcebos.com/80144f803a7b4d26b911de47f20956f9f5b8c77b0b9d4647b1d912758694286a)
![](https://ai-studio-static-online.cdn.bcebos.com/b6d09601627249de8c7d1a128cf945154a69c2f947fc468da5d298fd87c9ee06)
